<a href="https://colab.research.google.com/github/micah-shull/RAG-LangChain/blob/main/LC_009_RAG_CustServiceBot_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pip Install Packages

In [ ]:
!pip install --upgrade --quiet \
    langchain \
    langchain-huggingface \
    langchain-openai \
    langchain-community \
    chromadb \
    python-dotenv \
    transformers \
    accelerate \
    sentencepiece

## Load Libaries

In [ ]:
# 🌿 Environment setup
import os                                 # File paths and OS interaction
from dotenv import load_dotenv            # Load environment variables from .env file
import langchain; print(langchain.__version__)  # Check LangChain version

# 📄 Document loading and preprocessing
from langchain_core.documents import Document                   # Base document type
from langchain_community.document_loaders import TextLoader     # Loads plain text files
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Splits long docs into smaller chunks

# 🔢 Embeddings + vector storage
from langchain_huggingface import HuggingFaceEmbeddings         # HuggingFace embedding model
from langchain.vectorstores import Chroma                       # Persistent vector DB (Chroma)

# 💬 Prompting + output
from langchain_core.prompts import ChatPromptTemplate           # Chat-style prompt templates
from langchain_core.output_parsers import StrOutputParser       # Converts model output to string

# 🔗 Chains / pipelines
from langchain_core.runnables import Runnable, RunnableLambda   # Compose custom pipelines

# 🧠 (Optional) Hugging Face LLM client setup
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace  # For HF inference API

# 🧾 Pretty printing
import textwrap                         # Format long strings for printing
from pprint import pprint               # Nicely format nested data structures


0.3.25


## SET PARAMS

In [ ]:
from langchain_openai.chat_models.base import ChatOpenAI

# SET MODEL PARAMS
EMBED_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50
K = 2

# Load token from .env.
load_dotenv("/content/API_KEYS.env", override=True)

LLM_MODEL = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.4  # Moderate creativity; adjust as needed
)



## 🧾 Document Cleaning

### 🧾 1. **Load the `.txt` files**

We’ll loop through all files in the folder using `TextLoader`.

### 🧹 2. **Cleaning**

Basic cleaning (e.g. stripping newlines, extra whitespace) is often helpful **before splitting**, especially if the files came from exports or copy-paste.

### ✂️ 3. **Split into chunks**

We’ll use `RecursiveCharacterTextSplitter` to chunk documents (typically 500–1000 characters with slight overlap for context continuity).

---

### 🧼 Why Basic Cleaning Helps

* Removes linebreaks and blank lines that confuse LLMs
* Avoids splitting chunks in weird places
* Standardizes format before embedding

Later you can add more advanced cleaning (e.g., remove boilerplate, normalize headers), but this is a solid default.





In [ ]:
# Path to your documents
docs_path = "/content/CFFC_docs"

# Step 1: Load all .txt files in the folder
raw_documents = []
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(docs_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        raw_documents.extend(docs)

print(f"Loaded {len(raw_documents)} documents.")

# Step 2 (optional): Clean up newlines and extra whitespace
def clean_doc(doc: Document) -> Document:
    cleaned = " ".join(doc.page_content.split())  # Removes newlines & extra spaces
    return Document(page_content=cleaned, metadata=doc.metadata)

cleaned_documents = [clean_doc(doc) for doc in raw_documents]

# Step 3: Split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunked_documents = splitter.split_documents(cleaned_documents)

print(f"Split into {len(chunked_documents)} total chunks.")

# Preview the first 5 chunks
print(f"Showing first 5 of {len(chunked_documents)} chunks:\n")

for i, doc in enumerate(chunked_documents[:5]):
    print(f"--- Chunk {i+1} ---")
    print(f"Source: {doc.metadata.get('source', 'N/A')}\n")
    print(textwrap.fill(doc.page_content[:500], width=100))  # limit preview to 500 characters
    print("\n")

Loaded 7 documents.
Split into 174 total chunks.
Showing first 5 of 174 chunks:

--- Chunk 1 ---
Source: /content/CFFC_docs/CFFC_Gainesville Economic Indicators That Matter to Local Businesses.txt

Cashflow 4Cast Gainesville Economic Indicators That Matter to Local Businesses on April 02, 2025 📗
Gainesville Economic Indicators That Matter to Local Businesses 1. Average Weekly Earnings


--- Chunk 2 ---
Source: /content/CFFC_docs/CFFC_Gainesville Economic Indicators That Matter to Local Businesses.txt

to Local Businesses 1. Average Weekly Earnings (Gainesville) What It Is: This tracks the average
amount workers in Gainesville earn per week — across all private sector jobs. It’s one of the
clearest


--- Chunk 3 ---
Source: /content/CFFC_docs/CFFC_Gainesville Economic Indicators That Matter to Local Businesses.txt

all private sector jobs. It’s one of the clearest measures of take-home pay and gives insight into
what people can realistically afford. Why It Matters for Gainesville: When 

## ✅ Embed + Persist in Chroma




In [ ]:
# Step 1: Set up Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Step 2: Set up Chroma with persistence
persist_dir = "chroma_db"

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

print(f"✅ Stored {len(chunked_documents)} chunks in Chroma at '{persist_dir}'")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Stored 174 chunks in Chroma at 'chroma_db'


## ✅ Create the Retriever & Prompt Template

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": K})

# prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful assistant that uses business documents to answer questions.
Use the following context to answer the question as accurately as possible.

Context:
{context}

Question:
{question}

Answer:
""")


## ✅ Step 3: Create the RAG Chain & Run a Query!

In [ ]:
# Define RAG chain
rag_chain = (
    RunnableLambda(lambda d: {
        "question": d["question"],
        "docs": retriever.invoke(d["question"])
    })
    | RunnableLambda(lambda d: {
        "context": "\n\n".join([doc.page_content for doc in d["docs"]]),
        "question": d["question"]
    })
    | prompt_template
    | LLM_MODEL
    | StrOutputParser()
)

# Invoke RAG
response = rag_chain.invoke({
    "question": "What are the recent economic indicators in Gainesville that affect local businesses?"
})

# Print response nicely
import textwrap
print("\n" + textwrap.fill(response, width=100))



The recent economic indicators in Gainesville that affect local businesses are the local job market,
Gainesville's unemployment rate, and the overall pressure on the community. These indicators show a
shift in how people feel about the economy, which can have a ripple effect on local businesses even
if nothing internal has changed.


Before writing a refined prompt, let’s first define **guardrails** — these are behavioral, stylistic, and content-specific constraints that keep your RAG chatbot **on-brand, factual, and helpful**.

Because the goal is to deliver **accurate customer service responses** (about pricing, business benefits, product offering), here’s what we should lock down.

---

## ✅ Guardrails for RAG Assistant

### 1. 🧠 **Truthfulness / Grounding**

* ✅ "Only answer using the provided context."
* ✅ "If the answer is not found, say so clearly."
* ❌ No hallucinating product features, prices, or claims

### 2. 💬 **Style / Tone**

* ✅ "Keep a professional, informative tone."
* ✅ "Write in full sentences, avoid excessive marketing fluff."
* ❌ Don’t use overly casual or overly technical language unless requested

### 3. 📦 **Structure**

* ✅ “Use concise bullet points when listing benefits or pricing tiers.”
* ✅ “Highlight product value clearly before mentioning features.”

### 4. 📍 **Relevance / Locality**

* ✅ “Relate responses to business owners, startups, or small teams when appropriate.”
* ✅ “Avoid generic economic commentary unless specifically relevant to the product.”

### 5. 🚫 **Factual Safety Net**

* ✅ “Do not speculate or infer anything not stated in the documentation.”
* ✅ “Avoid offering recommendations that go beyond the scope of what the product or service supports.”





#TESTING

## 🧾 Prompt Template: Initial Version

This prompt defines the behavior and constraints for the Cashflow4cast RAG assistant, ensuring that responses are helpful, accurate, and grounded in business documentation.

### 🎯 Objective

Instruct the model to act as a **customer support assistant** for **Cashflow4cast**, a platform that helps businesses forecast cash flow using economic indicators.

### 🧠 Prompt Behavior

The assistant is expected to:

* **Answer based only on retrieved context documents**
* **Maintain a concise and professional tone**
* **Avoid making assumptions or hallucinations**

### 🧩 Key Instructions in the Prompt

* If the answer isn't found in the context, return:
  `"I'm sorry, I don't have that information based on the current documentation."`
* Structure responses clearly (e.g., use bullet points for lists like pricing or features)
* Focus only on what's **relevant to the business context**
* Exclude any **unsupported claims, personal opinions, or unrelated features**
* Do not fabricate pricing or services — only include what's explicitly in the documents




In [ ]:
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

prompt_template = ChatPromptTemplate.from_template("""
You are a customer support assistant for Cashflow4cast, a platform that helps business owners forecast cash flow using economic indicators.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- If the answer is not in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."
- Be concise, informative, and professional in tone.
- Structure answers clearly: use bullet points if listing pricing or benefits.
- Focus responses on business relevance. Do not include personal opinions or make assumptions.
- Do not mention features, services, or pricing unless explicitly included in the context.


Context:
{context}

Question:
{question}

Answer:
""")

# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)




🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
CashFlow4Cast uses advanced machine learning to help businesses cut forecasting errors in half,
providing clearer insight and earlier warnings for better cash flow forecasting.


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
The service works by providing businesses with a strong proxy for consumer demand and small business
health through data collected from retail employment numbers. This data can help businesses manage
their cash flow by giving them reliable numbers to make informed decisions regarding payroll,
inventory restocking, and preparing for seasonal fluctuations.


🔹 Test 3: What does it cost?
----------------------------------------------------------------------------------------------------
The cost for the Local Professional Services 



### 🔎 Review & Observations

#### **Test 1: What does Cashflow4cast actually do?**

✅ **Strengths**: Focuses on machine learning and forecasting improvements
⚠️ **Improvement**: Could include that the forecasts are powered by both *company data and economic indicators* for a more complete picture

---

#### **Test 2: How does the service work?**

⚠️ **Issue**: The mention of *retail employment numbers* feels too specific and might come from a misaligned chunk of context
✅ **Strengths**: It does stick to the broader idea of helping manage payroll, inventory, etc.

💡 **Suggestion**: Consider verifying that your context chunks don’t intermix examples from broader economic reports if those aren’t meant to represent the core product functionality.

---

#### **Test 3: What does it cost?**

❌ **Issue**: The answer about “Local Professional Services business type ranges from \$15,000 to \$40,000” is outdated or from a different document.

✅ **Fix already in progress**: You already addressed this by moving clean pricing info to the top of the pricing doc and adjusting chunk size. If you're still getting this, double-check if old chunks are being re-used from a previous embedding run (you may need to clear `persist_directory` before re-ingesting).

---

#### **Test 4: Can I use it for my small business?**

✅ **Strengths**: The answer is clear and affirmative
⚠️ **Tone**: Slightly generic (“use the information in the context”) — could sound more supportive and personalized

---

#### **Test 5: What makes Cashflow4cast different?**

⚠️ **Issue**: Reference to MAE and MAPE is too technical and doesn’t match your intended audience
✅ **Action already taken**: You’ve edited those out of the docs — once re-ingested, that language should drop off

---

### ✅ Recommendations Summary

| Area                 | Recommendation                                                                                                                               |
| -------------------- | -------------------------------------------------------------------------------------------------------------------------------------------- |
| **Context Content**  | Re-ingest documents with improved, simplified wording — especially pricing and metrics                                                       |
| **Chunk Validation** | Confirm you're loading only the updated `.txt` files and not leftover data                                                                   |
| **Prompt Tuning**    | Consider replacing strict tone constraints with: “friendly, helpful, and concise”                                                            |
| **Reasoning Chain**  | Consider adding a reasoning cue to the prompt, e.g., *“Take a moment to think through the question and look for the answer in the context.”* |



### 🔧 **Prompt Template Summary (Second Iteration)**

**Purpose**:
This prompt is designed to power a **customer support assistant** for Cashflow4cast, a financial forecasting platform for business owners.

**Tone & Behavior Goals**:

* **Professional** and **informative**
* **Concise** in response length
* **Context-grounded only** — no hallucinations or assumptions
* Avoids unnecessary speculation or off-topic commentary

**Instructions to the Assistant**:

* Only use information found in the provided business documents
* If the answer is missing, clearly state:
  *"I'm sorry, I don't have that information based on the current documentation."*
* Use **bullet points** for listing pricing, benefits, or features
* Do **not** fabricate or infer additional services, prices, or claims



In [ ]:
# SET MODEL PARAMS
EMBED_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
K = 4

# Path to your documents
docs_path = "/content/CFFC_docs"

# Step 1: Load all .txt files in the folder
raw_documents = []
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(docs_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        raw_documents.extend(docs)

print(f"Loaded {len(raw_documents)} documents.")

# Step 2 (optional): Clean up newlines and extra whitespace
def clean_doc(doc: Document) -> Document:
    cleaned = " ".join(doc.page_content.split())  # Removes newlines & extra spaces
    return Document(page_content=cleaned, metadata=doc.metadata)

cleaned_documents = [clean_doc(doc) for doc in raw_documents]

# Step 3: Split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunked_documents = splitter.split_documents(cleaned_documents)

# Step 1: Set up Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Step 2: Set up Chroma with persistence
persist_dir = "/content/chroma_db"  # In Colab, always use full paths

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

# define the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": K})

# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a customer support assistant for Cashflow4cast, a platform that helps business owners forecast cash flow using economic indicators.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- If the answer is not in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."
- Be concise, informative, and professional in tone.
- Structure answers clearly: use bullet points if listing pricing or benefits.
- Focus responses on business relevance. Do not include personal opinions or make assumptions.
- Do not mention features, services, or pricing unless explicitly included in the context.


Context:
{context}

Question:
{question}

Answer:
""")


print(f"✅ Stored {len(chunked_documents)} chunks in Chroma at '{persist_dir}'")

# Define RAG chain
rag_chain = (
    RunnableLambda(lambda d: {
        "question": d["question"],
        "docs": retriever.invoke(d["question"])
    })
    | RunnableLambda(lambda d: {
        "context": "\n\n".join([doc.page_content for doc in d["docs"]]),
        "question": d["question"]
    })
    | prompt_template
    | LLM_MODEL
    | StrOutputParser()
)

# Invoke RAG
response = rag_chain.invoke({
    "question": "What are the recent economic indicators in Gainesville that affect local businesses?"
})

# Print response nicely
import textwrap
print("\n" + textwrap.fill(response, width=100))



# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)

Loaded 7 documents.
✅ Stored 68 chunks in Chroma at '/content/chroma_db'

- Rising unemployment - Fewer retail jobs - Shrinking weekly paychecks

🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
- Cashflow4cast evaluates forecast performance using three key metrics: MAE, MAPE, and RMSE -
Cashflow4cast helps business owners manage cash flow by providing reliable forecasting tools


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
- Cashflow4cast uses a local machine learning model to forecast cash flow for different business
types - The model calculates revenue ranges and error percentages for each business type - Small
gains in forecasting accuracy can lead to real savings by enabling better ordering and smarter
staffing decisions


🔹 Test 3: What does it cost?
----------------------------------------



### 🔍 **Evaluation of Prompt Output (Second Iteration)**

#### 🔹 **Test 1: What does Cashflow4cast actually do?**

**✅ Strengths**:

* Points out use of forecasting metrics and focus on helping business owners.

**⚠️ Opportunities**:

* **Too technical** — MAE, MAPE, RMSE are not appropriate for a general customer support audience.
* The second bullet is generic; it could say more about *how* it helps.

**Suggested Fix**: Remove metric jargon from your source documents if the audience is non-technical. Emphasize outcomes (e.g., reduce forecasting error, improve cash flow clarity).

---

#### 🔹 **Test 2: How does the service work?**

**✅ Strengths**:

* Clear and process-oriented explanation.
* Mentions use of ML and practical benefits like staffing/order optimization.

**⚠️ Opportunities**:

* Could use slightly plainer language ("local machine learning model" might confuse some users).

**Suggestion**: Consider phrasing like *“a custom forecasting tool that uses economic data and your business history”*.

---

#### 🔹 **Test 3: What does it cost?**

**✅ Strengths**:

* Clear, concise, structured using bullet points.
* Includes all three plans and pricing.

**⚠️ Opportunities**:

* Minor: could briefly note differences between plans (e.g., who they’re best for).

**Optional**: Add “starting at” language to improve flexibility.

---

#### 🔹 **Test 4: Can I use it for my small business?**

**✅ Strengths**:

* Direct “Yes” answer, followed by examples of supported business types.

**⚠️ Opportunities**:

* Slightly robotic tone (“our Local Machine Learning Model”).

**Suggestion**: Rephrase to sound more conversational. E.g.,

> "Absolutely! Cashflow4cast supports small businesses like retail shops, cafés, and professional services."

---

#### 🔹 **Test 5: What makes Cashflow4cast different from other tools?**

**✅ Strengths**:

* Focuses on error reduction and advanced capabilities.

**⚠️ Opportunities**:

* Again, reliance on metrics (MAE, MAPE, RMSE) may confuse users.

**Suggestion**: Emphasize outcomes, not technical benchmarks.

---

### 🧠 Overall Assessment

| Aspect        | Result                                                                        |
| ------------- | ----------------------------------------------------------------------------- |
| **Clarity**   | ✅ Mostly clear, but some technical terms should be removed for this audience. |
| **Structure** | ✅ Bullet points are working well.                                             |
| **Tone**      | ⚠️ Slightly too formal/robotic in some responses. Can be friendlier.          |
| **Grounding** | ✅ On-topic and faithful to context.                                           |

---

### ✅ Actionable Fixes

1. **Remove or downplay technical jargon** (e.g., MAE, RMSE).
2. **Soften tone slightly** in prompt (e.g., “friendly and clear”).
3. **Optional**: Enhance pricing and feature responses with micro-descriptions for each tier.
4. **If needed**: Add example-based answers ("For example, a café might use this to...").




### 🔍 Response Review

## Modified Pricing Doc to Push Prices to the Top of Doc

#### 🔹 **Test 1: What does Cashflow4cast actually do?**

**✅ Pros**:

* Clear goal: reducing forecasting error.
* Emphasizes reliability in managing cash flow operations.

**⚠️ Issues**:

* **Still mentions MAE, MAPE, RMSE** — which conflicts with your earlier concern that the average user doesn't know these metrics.
* Slightly robotic bullet format for what could be a more fluid answer.

**💡 Suggestion**: Remove metric jargon from source doc and prompt the model to speak in outcomes:

> “Cashflow4cast helps you forecast cash flow more accurately—giving you clearer insight to manage payroll, restock inventory, and prep for slow seasons.”

---

#### 🔹 **Test 2: How does the service work?**

**✅ Pros**:

* Covers the mechanics well: ML + economic indicators + business data.
* Explains business impact: ordering, staffing, savings.

**⚠️ Issues**:

* The phrase "Local Machine Learning Model" is not user-friendly and may be unclear.
* Slightly repetitive of Test 1's answer in tone and structure.

**💡 Suggestion**: Encourage the model to phrase it more naturally:

> “Cashflow4cast analyzes both your business data and local economic signals to forecast revenue. This helps you make smarter decisions on staffing and inventory each week.”

---

#### 🔹 **Test 3: What does it cost?**

**✅ Pros**:

* Accurate.
* Clearly structured.

**⚠️ Issues**:

* Could include who each plan is best for (if that's in the doc).

**💡 Optional Enhancement**:

> “Choose the plan that fits your needs: Basic (\$199/mo), Advanced (\$499/mo), or Premium (\$999/mo) for real-time insights and expert consulting.”

---

#### 🔹 **Test 4: Can I use it for my small business?**

**✅ Pros**:

* Direct "Yes" answer.
* Provides relevant business types as examples.

**⚠️ Issues**:

* The phrase “Local Machine Learning Model” again feels opaque.
* Ends with a slightly technical tone (“low and high scenarios with corresponding error percentages”).

**💡 Suggestion**:

> “Yes! Cashflow4cast works well for small businesses like cafés, shops, and service providers—helping you forecast sales more confidently.”

---

#### 🔹 **Test 5: What makes Cashflow4cast different from other tools?**

**✅ Pros**:

* Emphasizes differentiator (50% error reduction).
* Speaks to accuracy and reliability.

**⚠️ Issues**:

* Repeats MAE/MAPE/RMSE — again too technical.
* Repeats earlier language too closely.

**💡 Suggestion**:

> “Unlike Excel or QuickBooks, Cashflow4cast blends your data with live economic trends, helping you predict cash flow more accurately and avoid surprises.”

---

### ✅ Summary Assessment

| Area               | Score                             | Notes                                             |
| ------------------ | --------------------------------- | ------------------------------------------------- |
| **Accuracy**       | ✅ Very good                       | Pulls relevant content cleanly.                   |
| **Tone**           | ⚠️ Neutral, slightly robotic      | Could benefit from softer, more natural phrasing. |
| **User Clarity**   | ⚠️ Technical terms still present  | MAE/MAPE/Local ML model could confuse users.      |
| **Consistency**    | ⚠️ Some repetition across answers | Vary structure/language slightly.                 |
| **Pricing Answer** | ✅ Accurate and clear              | Could optionally be enriched.                     |

---

### 📌 Final Tips

1. **Simplify source text**: You’ve already started doing this with pricing—now do the same for ML and metrics.
2. **Use friendly phrasing in your prompt**:

   * Replace "refer to the company documentation" with “use the info below to help the customer.”
   * Add tone instructions like: “Sound natural and helpful, like a real support rep.”



In [ ]:
# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful and friendly customer support assistant for Cashflow4cast.

Cashflow4cast: Forecasting You Can Trust in Uncertain Times

We leverage advanced machine learning and a combination of your business data and economic indicators \
including federal, state, and local metrics — to forecast future sales and cash flow more accurately than Excel, \
QuickBooks, or other off-the-shelf forecasting tools. Our ML models help mid-sized businesses stay ahead \
of sales volatility and protect their bottom line by cutting forecast errors in half.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- Read the question, refer to the company documentation, and provide a thoughtful, helpful, and informative response.
- Keep your answer under five sentences.
- Avoid repeating the same phrase across multiple answers unless it's directly relevant to the question.
- If the answer is not found in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."
- Do not include personal opinions or make assumptions.

Context:
{context}

Question:
{question}

Answer:
""")


# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

# Manual eval stub
def score_response(response, expected_keywords, verbose=True):
    score = sum(1 for kw in expected_keywords if kw.lower() in response.lower())
    if verbose:
        print(f"✅ Matched {score}/{len(expected_keywords)} keywords: {expected_keywords}")
    return score


# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)



🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
- Cashflow4cast evaluates forecast performance using three key metrics: MAE, MAPE, and RMSE -
Cashflow4cast helps business owners cut cash flow forecasting errors by 50% - Cashflow4cast provides
reliable numbers for managing cash flow, covering payroll, restocking inventory, and preparing for
seasonal dips


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
- Cashflow4cast uses a Local Machine Learning Model to forecast cash flow based on economic
indicators - The model provides revenue ranges and error percentages for different business types -
Small gains in forecasting accuracy can lead to real savings every month - Benefits include better
ordering and smarter staffing decisions


🔹 Test 3: What does it cost?
-----------------------------------------



### 🔧 **Technical Adjustments**

* **Chunk size reduced to 200**
  → Likely made responses more specific by narrowing each retrieval unit.
* **Overlap set to 50**
  → Maintains cohesion between chunks, useful for longer thoughts that span breaks.
* **K set to 2 (top 2 chunks)**
  → Forces the model to be more selective, reducing clutter or contradictions in context.

#### 📝 **Prompt Changes**

* **Added a warmer, branded introduction**
  → Personalizes the tone (“Forecasting You Can Trust in Uncertain Times”).
* **Softened tone instructions**
  → “Thoughtful, helpful, informative” now guides tone, vs. rigid formality.
* **Added constraints on verbosity and phrase reuse**
  → Keeps responses short and less robotic.
* **Maintained fallback behavior**
  → Still says “I don’t know” when context is missing (good guardrail).

---

### 📊 Review of the Outputs

| Test                              | Strengths                                                       | Suggested Improvements                                                     |
| --------------------------------- | --------------------------------------------------------------- | -------------------------------------------------------------------------- |
| **1**<br>What does it do?         | ✔️ On-message<br>✔️ Clear benefits                              | Could vary language to reduce repeated phrasing with Test 2.               |
| **2**<br>How does it work?        | ✔️ Mentions key inputs and mechanics<br>✔️ Describes outcomes   | First sentence is almost identical to Test 1 — maybe add a little variety. |
| **3**<br>What does it cost?       | ✔️ Correct info<br>✔️ Nicely condensed                          | None — very solid.                                                         |
| **4**<br>Small business?          | ✔️ Answers directly<br>✔️ Offers specific examples              | Could emphasize *why* it’s good for small businesses beyond type matching. |
| **5**<br>What makes it different? | ✔️ Strong differentiator focus<br>✔️ Less technical than before | Consider using a metaphor or casual phrasing to feel more personal.        |

---

### 🎯 Overall Feedback

* ✅ **Much improved tone**: Clear, warm, and human without being verbose.
* ✅ **Pricing now accurate and easy to read**.
* ✅ **No more confusing jargon (MAE/MAPE/etc.)** — huge win.
* ⚠️ **Repetition** between Test 1 and 2 is your biggest lingering issue.
* 🧠 **Persona alignment is excellent**: The assistant now genuinely feels like a branded support rep.

---

### ✅ Suggested Small Next Steps

1. **Diversify phrasing**: Add to your prompt something like:

   > “Try to phrase each answer naturally and avoid repeating entire sentences unless necessary.”

2. **Highlight “why it matters”** in small biz or competitive advantage answers — make it feel more “human to human.”

3. **Optional**: Tag each test question with expected intent (e.g., *feature discovery*, *sales objection*, *trust validation*) for future prompt tuning or A/B testing.



In [ ]:
# SET MODEL PARAMS
EMBED_MODEL = "all-MiniLM-L6-v2"
CHUNK_SIZE = 200
CHUNK_OVERLAP = 50
K = 2

# Path to your documents
docs_path = "/content/CFFC_docs"

# Step 1: Load all .txt files in the folder
raw_documents = []
for filename in os.listdir(docs_path):
    if filename.endswith(".txt"):
        file_path = os.path.join(docs_path, filename)
        loader = TextLoader(file_path, encoding="utf-8")
        docs = loader.load()
        raw_documents.extend(docs)

print(f"Loaded {len(raw_documents)} documents.")

# Step 2 (optional): Clean up newlines and extra whitespace
def clean_doc(doc: Document) -> Document:
    cleaned = " ".join(doc.page_content.split())  # Removes newlines & extra spaces
    return Document(page_content=cleaned, metadata=doc.metadata)

cleaned_documents = [clean_doc(doc) for doc in raw_documents]

# Step 3: Split documents into chunks
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)

chunked_documents = splitter.split_documents(cleaned_documents)

# print(f"Split into {len(chunked_documents)} total chunks.")

# # Preview the first 5 chunks
# print(f"Showing first 5 of {len(chunked_documents)} chunks:\n")

# for i, doc in enumerate(chunked_documents[:5]):
#     print(f"--- Chunk {i+1} ---")
#     print(f"Source: {doc.metadata.get('source', 'N/A')}\n")
#     print(textwrap.fill(doc.page_content[:500], width=100))  # limit preview to 500 characters
#     print("\n")

# Step 1: Set up Hugging Face embedding model
embedding_model = HuggingFaceEmbeddings(model_name=EMBED_MODEL)

# Step 2: Set up Chroma with persistence
persist_dir = "/content/chroma_db"  # In Colab, always use full paths

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embedding_model,
    persist_directory=persist_dir
)

# define the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": K})

# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful and friendly customer support assistant for Cashflow4cast.

Cashflow4cast: Forecasting You Can Trust in Uncertain Times

We leverage advanced machine learning and a combination of your business data and economic indicators — including federal, state, and local metrics — to forecast future sales and cash flow more accurately than Excel, QuickBooks, or other off-the-shelf forecasting tools.

Our platform helps mid-sized businesses stay ahead of sales volatility and protect their bottom line by cutting forecast errors in half.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- Read the question, refer to the company documentation, and provide a thoughtful, helpful, and informative response.
- Keep your answer under five sentences.
- Avoid repeating the same phrase across multiple answers unless it's directly relevant to the question.
- If the answer is not found in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."
- Do not include personal opinions or make assumptions.

Context:
{context}

Question:
{question}

Answer:
""")

print(f"✅ Stored {len(chunked_documents)} chunks in Chroma at '{persist_dir}'")

# Define RAG chain
rag_chain = (
    RunnableLambda(lambda d: {
        "question": d["question"],
        "docs": retriever.invoke(d["question"])
    })
    | RunnableLambda(lambda d: {
        "context": "\n\n".join([doc.page_content for doc in d["docs"]]),
        "question": d["question"]
    })
    | prompt_template
    | LLM_MODEL
    | StrOutputParser()
)

# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)


🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of business data and economic
indicators to forecast future sales and cash flow more accurately than traditional tools like Excel
or QuickBooks. This helps mid-sized businesses stay ahead of sales volatility and reduce forecast
errors by half, ultimately protecting their bottom line.


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of your business data and
economic indicators to forecast future sales and cash flow more accurately than traditional tools
like Excel or QuickBooks. By analyzing federal, state, and local metrics, our platform helps mid-
sized businesses anticipate sales volatility and reduce forecast err



## 📝 Summary of Prompt Changes

**Changed:**

* Shifted tone to *“friendly and knowledgeable”* rather than just “customer support assistant”
* Added a short branded summary:

  > “Cashflow4cast helps business owners forecast sales and cash flow more accurately…”
* Replaced rigid instructions with natural, human ones:

  * “Consider the question carefully”
  * “Review the documentation”
  * “Compose a clear and helpful response”
* Emphasized:

  * Tailoring to the question
  * Avoiding unnecessary repetition
  * Using short paragraphs or bullet points
  * Clear fallback when context is missing

**Effect:**
✅ More natural and approachable tone
✅ Gentler scaffolding that still provides structure
✅ Encourages thoughtful, context-aware answers

---

## 🔍 Output Review

| Test                     | ✅ Strengths                                                | ⚠️ Suggestions                                  |
| ------------------------ | ---------------------------------------------------------- | ----------------------------------------------- |
| **1** What does it do?   | ✅ Clear and accurate<br>✅ Avoids jargon                    | 🔁 Still repeats language from other answers    |
| **2** How does it work?  | ✅ Explains with layered info<br>✅ Covers process           | 🔁 Opening is a near match to Test 1            |
| **3** Pricing            | ✅ Clear breakdown by tier                                  | 💬 Consider using bullet points here too        |
| **4** Small business use | ✅ Direct, friendly, business-relevant<br>✅ Great tone      | 🔁 Same “Excel or QuickBooks” phrase as earlier |
| **5** Differentiator     | ✅ Well-framed value prop<br>✅ Simple, grounded explanation | ✅ Strongest answer — no changes needed          |

---

## 🧠 Overall Impression

You’re now in the **“polishing” phase** — the structure and information delivery are excellent. The tone has improved, the fallback logic is intact, and the assistant sounds **clear, competent, and friendly**.

---

## ✅ Suggested Minor Enhancements

1. **Add this to your prompt to reduce repetition**:

   > Avoid repeating full phrases from previous answers unless the exact phrasing is essential for clarity.

2. **Swap "Excel or QuickBooks" references** in a few responses:

   * Try phrases like “manual spreadsheets,” “accounting software,” or “standard forecasting tools.”

3. **Make pricing answer more skimmable**:

   * Use bullet points:

     ```
     - Basic: $199/mo
     - Advanced: $499/mo
     - Premium: Custom pricing for real-time needs
     ```



In [ ]:
# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a friendly and knowledgeable customer support assistant for Cashflow4cast.

Cashflow4cast helps business owners forecast sales and cash flow more accurately using machine learning and real-world economic indicators. \
Our goal is to reduce forecast errors by half, helping companies stay ahead of volatility and protect their bottom line.

Answer the user's question using the information provided in the business documents below.

Instructions:
- Consider the question carefully.
- Review the provided documentation for the most relevant information.
- Then compose a clear and helpful response based only on the context.

- Be clear, kind, and helpful.
- Tailor your answer to the specific question — don’t repeat generic company information unless it's directly relevant.
- Use short paragraphs and bullet points when helpful.
- If the answer isn't in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."

Context:
{context}

Question:
{question}

Answer:
""")


# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]

# Manual eval stub
def score_response(response, expected_keywords, verbose=True):
    score = sum(1 for kw in expected_keywords if kw.lower() in response.lower())
    if verbose:
        print(f"✅ Matched {score}/{len(expected_keywords)} keywords: {expected_keywords}")
    return score


# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)



🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of business data and economic
indicators to forecast future sales and cash flow more accurately than traditional tools like Excel
or QuickBooks. Their platform helps mid-sized businesses stay ahead of sales volatility and reduce
forecast errors by half.


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of your business data and
economic indicators to forecast future sales and cash flow more accurately than traditional tools
like Excel or QuickBooks. By analyzing federal, state, and local metrics, our platform helps mid-
sized businesses predict sales volatility and protect their bottom line by reducing forecast errors



### 🔧 What Changed in the Prompt

**Updated Intro:**

* Replaced branding block with a clear mission-focused paragraph:

  > “Cashflow4cast leverages advanced machine learning algorithms that utilize your business data…”

**Refined Instructions:**

* Simplified the assistant’s role:
  *“You are a helpful and friendly customer support assistant...”*
* Emphasized a three-step reasoning approach:

  * Consider → Review → Compose
* Introduced new constraint:
  *“Keep your answer under four sentences.”*

**Tone Goals:**

* More approachable and conversational
* Less instructional overhead
* Still rooted in clarity and professionalism

---

## ✅ Output Review

| Test                   | ✅ Strengths                                                              | ⚠️ Suggestions                                                  |
| ---------------------- | ------------------------------------------------------------------------ | --------------------------------------------------------------- |
| **1** What does it do? | ✅ Tight summary<br>✅ Clear differentiation from generic tools            | 🔁 Repeats same “advanced machine learning…” phrasing as others |
| **2** How it works     | ✅ Logical flow from tech → benefit<br>✅ Strong value claim               | 🔁 Shares \~80% word-for-word with Test 1                       |
| **3** Pricing          | ✅ Accurate pricing info<br>✅ Clear, brief, no fluff                      | 📌 Could use bullet points for even faster scanning             |
| **4** Small business   | ✅ Excellent relevance & tone<br>✅ Solid example types listed             | 🔁 “Leverages machine learning…” repeated again                 |
| **5** Differentiation  | ✅ Strong contrast with “off-the-shelf tools”<br>✅ Hits unique value prop | 🔁 Ends with a generic comparison (can soften repetition)       |

---

## 🔍 Patterns + Recommendations

### 🟡 Minor Repetition

* **The phrase** “leverages advanced machine learning and a combination of business data and economic indicators…” appears in almost every answer.
* 🛠 *Fix:* Add to the prompt:

  > “Avoid repeating the same phrase across multiple answers unless it's directly relevant.”

### 📌 Structure Opportunities

* Use **bullet points** for pricing (Test 3) and differentiation (Test 5) to improve skimmability.

### 💡 Enhancement Suggestion for Prompt

Here’s an updated version of your prompt to incorporate these learnings:

```python
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful and friendly customer support assistant for Cashflow4cast.

Cashflow4cast helps business owners forecast sales and cash flow using machine learning and real-world economic indicators.
We help mid-sized businesses reduce forecast errors, stay ahead of volatility, and protect their bottom line.

Answer the user's question using only the information provided in the business documents below.

Instructions:
- Consider the question carefully.
- Review the documentation to find the most relevant answer.
- Then compose a clear and helpful response — under four sentences.
- Use bullet points when listing options like pricing or features.
- Avoid repeating identical phrasing across multiple answers.
- If the answer is not found in the provided context, say: "I'm sorry, I don't have that information based on the current documentation."
""")
```



In [ ]:
# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
You are a helpful and friendly customer support assistant for Cashflow4cast.

Cashflow4cast leverages advanced machine learning algorithms that utlize your business data and combine that \
with federal, state, and local economic indicators — to forecast future sales and cash flow more accurately than Excel, \
QuickBooks, or any other off-the-shelf forecasting tools. We help mid-sized businesses stay ahead of sales volatility \
increae profit and reduce inventory by cutting forecast errors in half.

Your task it to nswer the user's question below:

- Consider the question carefully.
- Review the provided documentation for the most relevant information.
- Then compose a clear and helpful response.
- Keep your answer under four sentences.

Context:
{context}

Question:
{question}

Answer:
""")


# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]


# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)


🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of business data and economic
indicators to forecast future sales and cash flow more accurately than Excel, QuickBooks, or other
off-the-shelf tools. This helps mid-sized businesses stay ahead of sales volatility and reduce
forecast errors by half.


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of your business data and
economic indicators to forecast future sales and cash flow more accurately than traditional tools
like Excel or QuickBooks. By analyzing federal, state, and local metrics, our platform helps mid-
sized businesses stay ahead of sales volatility and protect their bottom line by cutting forecast
errors



## ✅ **What Worked Best**

### 1. **Simplified, Friendly Prompt Design**

* **Best-performing prompt** avoided excessive structure and let the model rely on its helpful, conversational training.
* Clearly stated *role* (customer support assistant) and *task* (answer based on docs).
* Friendly and approachable tone resonated more naturally than rigidly professional language.

### 2. **Shorter Chunks + Lower `k`**

* `CHUNK_SIZE = 200` with `K = 2` struck a great balance.
* Delivered *high precision* answers grounded in relevant context, minimizing hallucination.
* Moving pricing info to the **top of the doc** improved its recall.

### 3. **Four-Sentence Limit**

* Helped keep answers snappy and digestible.
* Reduced the model’s tendency to over-explain or repeat itself.

### 4. **Instruction to Avoid Repeating Phrases**

* This lowered redundancy and made answers feel more tailored and less templated.

---

## ⚠️ **What Didn't Work Well**

### 1. **Overly Long or Formal Prompts**

* Prompts that tried to “over-steer” the model with excessive behavioral rules or detailed instructions led to:

  * Stiffer, less natural tone
  * Redundant answers
  * Unneeded technical language (e.g., MAE, MAPE, RMSE)

### 2. **Large Chunks + High `k`**

* Caused diffusion of relevant info
* Introduced outdated pricing and overly technical language even after doc updates

---

## 💡 **Recommendations Going Forward**

### 🔧 Prompt

Use a prompt that is:

* Clear in role and purpose
* Friendly and natural
* Lightly instructive (not overbearing)

**Example Baseline Prompt:**

```python
"You are a helpful and friendly customer support assistant for Cashflow4cast.
Use only the context below to answer the question. Keep your response under four sentences.
Avoid repeating the same phrase across answers. If the info isn't in the context, say:
'I'm sorry, I don't have that information based on the current documentation.'"
```

### 📐 Parameters

* `CHUNK_SIZE = 200`
* `CHUNK_OVERLAP = 50`
* `K = 2`

This setup optimizes *focus, relevance,* and *efficiency*.

### 🧪 Testing

* Keep using your test suite of 5–7 customer questions.
* Add `score_response()` to track keyword or intent coverage.
* Track repeat phrases and measure tone drift across runs.

### 📘 Documentation

Consider adding a markdown section to your notebook or repo summarizing:

* Prompt iterations
* Chunking/embedding settings
* Sample outputs + ratings

---

## ✅ Final Takeaway

You've done the hard work of fine-tuning a prompt for **business relevance**, **user experience**, and **model strengths**. The winning formula? Trust the model’s defaults, provide clear and gentle guidance, and ensure the RAG context is crisp and well-prioritized.



In [ ]:
# create prompt template
prompt_template = ChatPromptTemplate.from_template("""
"You are a helpful and friendly customer support assistant for Cashflow4cast.
Use only the context below to answer the question. Keep your response under four sentences.
Avoid repeating the same phrase across answers. If the info isn't in the context, say:
'I'm sorry, I don't have that information based on the current documentation.'"

Context:
{context}

Question:
{question}

Answer:
""")


# Define test questions
test_questions = [
    "What does Cashflow4cast actually do?",
    "How does the service work?",
    "What does it cost?",
    "Can I use it for my small business?",
    "What makes Cashflow4cast different from other tools?"
]


# Define test runner
def run_prompt_tests(questions, rag_chain, wrap_width=100):
    for i, question in enumerate(questions, 1):
        print(f"\n🔹 Test {i}: {question}\n{'-'*wrap_width}")
        response = rag_chain.invoke({"question": question})
        print(textwrap.fill(response, width=wrap_width))
        print("\n" + "="*wrap_width)

# Run test questions
run_prompt_tests(test_questions, rag_chain)


🔹 Test 1: What does Cashflow4cast actually do?
----------------------------------------------------------------------------------------------------
Cashflow4cast uses advanced machine learning and a combination of business data and economic
indicators to forecast future sales and cash flow more accurately than Excel, QuickBooks, or other
off-the-shelf tools. Their platform helps mid-sized businesses stay ahead of sales volatility and
reduce forecast errors by half.


🔹 Test 2: How does the service work?
----------------------------------------------------------------------------------------------------
Cashflow4cast leverages advanced machine learning and a combination of your business data and
economic indicators to forecast future sales and cash flow more accurately than traditional tools
like Excel or QuickBooks. By analyzing federal, state, and local metrics, our platform helps mid-
sized businesses stay ahead of sales volatility and protect their bottom line by reducing forecast


These results are excellent — you're in a really solid place. Here's a
### ✅ **What’s Working Well**

#### 1. **Clarity + Accuracy**

* Each answer directly addresses the question using *precise* and *business-relevant* language.
* There's no hallucination or fluff, and pricing is **correct** and consistent.

#### 2. **Tone**

* Answers feel **clear, confident, and professional**, yet friendly — exactly the tone you were aiming for.
* No excessive formalism or robotic phrasing.

#### 3. **Length & Structure**

* All responses are comfortably under 4 sentences, with **tight, informative phrasing**.
* Test 4 even varies structure slightly (second sentence adds user-oriented detail), showing good balance.

---

### ⚠️ **Tiny Refinements (Optional)**

#### 🔁 **Slight Redundancy in Phrasing**

Many responses start with:

> "Cashflow4cast leverages advanced machine learning and a combination of your business data and economic indicators..."

This repetition across Test 1, 2, and 5 could be slightly fatiguing over time. A minor adjustment like:

* “Our platform combines your business data with real-world economic signals…”
* “We use intelligent forecasting models built on local and national data…”

…could add variety without changing substance.

#### 🧠 **User Empathy in Small Business Q (Test 4)**

It’s great as-is, but optionally you could soften the phrasing for small businesses even more:

> “Absolutely. Cashflow4cast is designed to support small businesses like yours...”

—

### ✅ **Verdict**

You're getting *clean, business-relevant, tone-appropriate answers* — the goal of any customer support RAG system. This setup is production-ready for most applications.

---

### 💡 Final Tip

You can preserve this success by:

* Freezing this prompt and chunk config in version control
* Logging responses and reviewing periodically for drift
* Collecting real customer questions over time to expand test coverage


In [2]:
import json
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

notebook_path ="/content/drive/My Drive/LANGCHAIN/LC_009_RAG_CustServiceBot_Testing.ipynb"

# Load the notebook JSON
with open(notebook_path, 'r', encoding='utf-8') as f:
    nb = json.load(f)

# 1. Remove widgets from notebook-level metadata
if "widgets" in nb.get("metadata", {}):
    del nb["metadata"]["widgets"]
    print("✅ Removed notebook-level 'widgets' metadata.")

# 2. Remove widgets from each cell's metadata
for i, cell in enumerate(nb.get("cells", [])):
    if "metadata" in cell and "widgets" in cell["metadata"]:
        del cell["metadata"]["widgets"]
        print(f"✅ Removed 'widgets' from cell {i}")

# Save the cleaned notebook
with open(notebook_path, 'w', encoding='utf-8') as f:
    json.dump(nb, f, indent=2)

print("✅ Notebook deeply cleaned. Try uploading to GitHub again.")

Mounted at /content/drive
✅ Notebook deeply cleaned. Try uploading to GitHub again.
